# HVM Large-Amplitude Oscillatory Shear (LAOS)

**Hybrid Vitrimer Model — Lissajous curves and TST harmonic generation**

## Physical Motivation

LAOS probes **nonlinear viscoelastic behavior** by imposing strains large enough
to activate the TST mechanism. During an oscillation cycle, stress varies
between $\pm\sigma_{max}$, periodically accelerating bond exchange via
$k_{BER} \propto \cosh(V_{act} \sigma_{VM}^E / RT)$. This creates a nonlinear
feedback loop between stress and microstructural dynamics that generates
**odd harmonics** ($3\omega$, $5\omega$, ...) in the stress response.

LAOS is particularly powerful for vitrimers because:
- **I$_3$/I$_1$ ratio** fingerprints the TST activation mechanism
- **Lissajous curves** reveal intracycle yielding and recovery
- **Amplitude sweeps** determine the onset of nonlinearity ($\gamma_c$)
- **Normal stress $N_1$** oscillates at $2\omega$ due to finite strain coupling

> **Handbook:** See [HVM Protocol Derivations](../../docs/source/models/hvm/hvm_protocols.rst) for the LAOS ODE system and harmonic extraction. See [HVM Advanced Theory](../../docs/source/models/hvm/hvm_advanced.rst) for the thermodynamic basis of nonlinear stress response.

## Learning Objectives

- Visualize elastic and viscous Lissajous curves
- Understand harmonic generation from TST-activated bond exchange
- Observe $N_1$ (first normal stress difference) at $2\omega$ frequency
- Analyze amplitude-dependent nonlinearity via $I_3/I_1$

## Prerequisites

- **Notebook 01** (SAOS) — HVM architecture, TST kinetics
- **Notebook 03** (Startup) — TST overshoot mechanism

## Estimated Runtime

- ~3 minutes

In [ ]:
# Google Colab setup (run only if on Colab)
try:
    import google.colab
    IN_COLAB = True
    !pip install -q rheojax
except ImportError:
    IN_COLAB = False
    print("Running locally")

In [ ]:
# Imports
from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.models.hvm import HVMLocal

plt.rcParams.update({
    'figure.dpi': 100,
    'font.size': 10,
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'legend.fontsize': 9
})

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
import os
import sys

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

## Theory: LAOS and TST Nonlinearity

In large-amplitude oscillatory shear (LAOS), the strain is:

$$\gamma(t) = \gamma_0 \sin(\omega t)$$

For the HVM model, nonlinearity arises from:

1. **TST stress dependence**: The E-network bond exchange rate depends on stress via k_ex(σ) ∼ exp(V_act·σ/(RT))
2. **Natural state evolution**: The reference configuration evolves nonlinearly with deformation
3. **Finite strain effects**: Large deformations couple stress components nonlinearly

The stress response contains odd harmonics:

$$\sigma(t) = \sum_{n=1,3,5,...} [G'_n \sin(n\omega t) + G''_n \cos(n\omega t)]$$

The third harmonic intensity I₃/I₁ quantifies nonlinearity. For small amplitudes, I₃/I₁ → 0 and the response becomes linear.

**Lissajous curves** plot:
- **Elastic**: σ vs γ (ellipse → distorted loop)
- **Viscous**: σ vs γ̇ (ellipse → distorted loop)

In [ ]:
# Model setup with strong TST coupling
model = HVMLocal(kinetics="stress")
model.parameters.set_value("G_P", 5000.0)    # Permanent network modulus [Pa]
model.parameters.set_value("G_E", 3000.0)    # E-network (exchangeable) [Pa]
model.parameters.set_value("G_D", 1000.0)    # D-network (dangling) [Pa]
model.parameters.set_value("T", 350.0)       # Temperature [K]
model.parameters.set_value("k_d_D", 10.0)    # D-network dissociation rate [1/s]
model.parameters.set_value("nu_0", 1e10)     # TST attempt frequency [1/s]
model.parameters.set_value("E_a", 80e3)      # Activation energy [J/mol]
model.parameters.set_value("V_act", 1e-4)    # Activation volume - INCREASED for TST

print("HVM Model Parameters:")
print(f"  G_P = {model.G_P:.0f} Pa")
print(f"  G_E = {model.G_E:.0f} Pa")
print(f"  G_D = {model.G_D:.0f} Pa")
print(f"  T = {model.T:.0f} K")
print(f"  V_act = {model.V_act:.2e} m³/mol (strong TST coupling)")


In [ ]:
# LAOS simulation
omega = 1.0
gamma_0 = 0.5
n_cycles = 10
t = np.linspace(0, n_cycles * 2 * np.pi / omega, n_cycles * 200)

print(f"Simulating LAOS: γ₀ = {gamma_0}, ω = {omega:.1f} rad/s, {n_cycles} cycles...")
result = model.simulate_laos(t, gamma_0=gamma_0, omega=omega)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Elastic Lissajous (σ vs γ)
ax1.plot(result["strain"], result["stress"], "k-", lw=0.5, alpha=0.3)
# Last cycle only (highlighted)
n_per = int(len(t) / n_cycles)
ax1.plot(result["strain"][-n_per:], result["stress"][-n_per:], "b-", lw=2)
ax1.set_xlabel("Strain γ")
ax1.set_ylabel("Stress σ [Pa]")
ax1.set_title("Elastic Lissajous Curve (last cycle)")
ax1.grid(True, alpha=0.3)
ax1.axhline(0, color="k", lw=0.5, alpha=0.5)
ax1.axvline(0, color="k", lw=0.5, alpha=0.5)

# Viscous Lissajous (σ vs γ̇)
gamma_dot_t = gamma_0 * omega * np.cos(omega * t)
ax2.plot(gamma_dot_t[-n_per:], result["stress"][-n_per:], "r-", lw=2)
ax2.set_xlabel("Strain rate γ̇ [1/s]")
ax2.set_ylabel("Stress σ [Pa]")
ax2.set_title("Viscous Lissajous Curve (last cycle)")
ax2.grid(True, alpha=0.3)
ax2.axhline(0, color="k", lw=0.5, alpha=0.5)
ax2.axvline(0, color="k", lw=0.5, alpha=0.5)

plt.tight_layout()
display(fig)
plt.close(fig)

print(f"\nStress range: {result['stress'].min():.2f} to {result['stress'].max():.2f} Pa")

In [ ]:
# Harmonic extraction
print("\nExtracting Fourier harmonics from LAOS stress signal...")
harmonics = model.extract_laos_harmonics(result, n_harmonics=5)

harmonic_idx = harmonics["harmonic_index"]
sigma_amps = harmonics["sigma_harmonics"]

print("\nFourier Harmonics (stress):")
print(f"{'n':>3} {'|I_n|':>12} {'I_n/I_1':>12}")
print("-" * 30)
amp_1 = sigma_amps[0] if sigma_amps[0] > 0 else 1.0
for n, amp in zip(harmonic_idx, sigma_amps):
    ratio = float(amp) / float(amp_1)
    print(f"{int(n):>3} {float(amp):>12.2f} {ratio:>12.4f}")

if len(sigma_amps) >= 2 and sigma_amps[0] > 0:
    nonlinearity_ratio = float(sigma_amps[1]) / float(sigma_amps[0])
    print(f"\nNonlinearity ratio I₃/I₁ = {nonlinearity_ratio:.4f}")
    if nonlinearity_ratio < 0.01:
        print("→ Nearly linear response (I₃/I₁ < 1%)")
    elif nonlinearity_ratio < 0.1:
        print("→ Weakly nonlinear response (1% < I₃/I₁ < 10%)")
    else:
        print("→ Strongly nonlinear response (I₃/I₁ > 10%)")

In [ ]:
# Amplitude sweep: compare small vs large deformation
amplitudes = [0.01, 0.1, 0.5, 1.0]
colors = ['blue', 'green', 'orange', 'red']

print("\nAmplitude sweep: comparing γ₀ = [0.01, 0.1, 0.5, 1.0]...")
fig, axes = plt.subplots(1, len(amplitudes), figsize=(4*len(amplitudes), 4))

for ax, g0, color in zip(axes, amplitudes, colors):
    r = model.simulate_laos(t, gamma_0=g0, omega=omega)
    n_per = int(len(t) / n_cycles)
    ax.plot(r["strain"][-n_per:], r["stress"][-n_per:], "-", lw=1.5, color=color)
    ax.set_xlabel("γ")
    ax.set_title(f"γ₀ = {g0}")
    ax.grid(True, alpha=0.3)
    ax.axhline(0, color="k", lw=0.5, alpha=0.3)
    ax.axvline(0, color="k", lw=0.5, alpha=0.3)
    
    # Annotate with harmonic ratio
    h = model.extract_laos_harmonics(r, n_harmonics=5)
    h_amps = h["sigma_harmonics"]
    if len(h_amps) >= 2 and h_amps[0] > 0:
        ratio = float(h_amps[1]) / float(h_amps[0])
        ax.text(0.05, 0.95, f"I₃/I₁ = {ratio:.3f}", 
                transform=ax.transAxes, va='top', fontsize=9,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

axes[0].set_ylabel("σ [Pa]")
plt.suptitle("Amplitude Sweep: Small → Large Deformation", fontsize=12, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

print("\nObserve: Lissajous curves transition from ellipse (linear) to distorted loops (nonlinear)")

## Key Takeaways

1. **TST generates nonlinearity**: Stress-dependent bond exchange rate produces odd harmonics in LAOS
2. **$I_3/I_1$ quantifies nonlinearity**: Third-to-fundamental harmonic ratio increases with amplitude
3. **Small-amplitude limit**: $\gamma_0 \to 0$ gives linear ellipse (recovers classical SAOS)
4. **Large-amplitude distortion**: Lissajous curves become asymmetric due to TST activation
5. **$V_{act}$ controls nonlinearity onset**: Materials with strong mechanochemical coupling show nonlinearity at lower $\gamma_0$

## Further Reading

- [HVM Model Reference](../../docs/source/models/hvm/hvm.rst) — Protocol summary and TST kinetics
- [HVM Protocol Derivations](../../docs/source/models/hvm/hvm_protocols.rst) — LAOS harmonic analysis

**References:**
1. Vernerey, F.J., Long, R. & Brighenti, R. (2017). *J. Mech. Phys. Solids*, 107, 1-20.
2. Hyun, K. et al. (2011). "A review of nonlinear oscillatory shear tests." *Prog. Polym. Sci.*, 36, 1697-1753.

## Next Notebooks

- **Notebook 07**: HVM Overview — model architecture and limiting cases
- **Notebooks 08-13**: Full NLSQ + NUTS Bayesian inference pipeline